In [2]:
%load_ext autoreload
%autoreload 2

from pathlib import Path

import pandas as pd

from impreproc.images import Image, ImageList
from impreproc.renaming import ImageRenamer

# Define parameters
# Note that in notebooks, the path are relative to the notebooks folder!
data_dir = Path("../data/renaming")
image_ext = ["jpg"]
dest_folder = Path("../res/renamed")
base_name = "IMG"
recursive = True
delete_original = False
overlay_name = True
parallel = True

# Get list of files
files = ImageList(data_dir, image_ext=image_ext, recursive=recursive)

Fast rename   

In [2]:
# Create ImageRenamer object
renamer = ImageRenamer(
    image_list=files,
    dest_folder=dest_folder,
    base_name=base_name,
    delete_original=delete_original,
    parallel=parallel,
    )
renamer.rename_fast()

33it [00:00, 79.03it/s]


True

Rename with overlay and saving output to pd df

In [3]:
# Create ImageRenamer object
renamer = ImageRenamer(
    image_list=files,
    dest_folder=dest_folder,
    base_name=base_name,
    delete_original=delete_original,
    overlay_name=overlay_name,
    parallel=parallel,
)

# Rename files
dic = renamer.rename()

# Save renaming dictionary
df = pd.DataFrame.from_dict(dic, orient="index")
df.to_csv(dest_folder/"renaming_dict.csv", index=False)


33it [00:03, 10.99it/s]


In [29]:
# Test for img calibration

from pprint import pprint

id = 0
img = Image(files[id])
print(img.exif["Image Model"])
K = img.get_intrinsics_from_exif()
pprint(K)


NIKON D800
array([[4.10027855e+03, 0.00000000e+00, 3.68000000e+03],
       [0.00000000e+00, 4.10027855e+03, 2.45600000e+03],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])
